In [1]:
#Import the required libraries
library(tidyverse)
library(repr)
library(tidymodels)
library(gridExtra)   
library(patchwork)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

Warning message:
“package ‘ggplot2’ was built under R version 4.1.3”
Warning message:
“package ‘tidyr’ was built under R version 4.1.2”
Warning message:
“package ‘readr’ was built under R version 4.1.2”
Warning message:
“package ‘dplyr’ was built under R version 4.1.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.1.3”
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ pars

ERROR: Error in library(gridExtra): there is no package called ‘gridExtra’


### We will use age, trestbps(Blood pressure), and max_heart_rate(Maximum Heart Rate). 

In order to predict if there exists a heart disease in a person, we will use three parameters. Age of the person, blood pressure and their maximum heart rate. We believe that a good idea to start to see how these different parameters are related to others, is to plot them against each other. So we created three different graphs to showcase this.

// *IF THIS DOESNT WORK OR THERE IS ANOTHER INTRO THEN DELETE* //

Code to tidy data set:

In [2]:
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
colmns_names <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restceg", "max_heart_rate",
               "exang", "oldpeak", "slope", "ca", "thal", "heart_disease")


heart_data <- read_csv(url, col_names = colmns_names)|>
    #Change the data type in the columns to the necessary data type for our classification
    mutate(across(c(trestbps, fbs, restceg, max_heart_rate, exang, oldpeak, slope, ca, thal),
                as.numeric),
          heart_disease = ifelse(heart_disease == "1" | heart_disease == "2" | 
                                 heart_disease == "3" | heart_disease == "4", "True", heart_disease),
          heart_disease = ifelse(heart_disease == "0", "False", heart_disease),
          heart_disease = as.factor(heart_disease))
# Preview the first 10 observations
head(heart_data, n=10)

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): ca, thal
dbl (12): age, sex, cp, trestbps, chol, fbs, restceg, max_heart_rate, exang,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”
Warning message in mask$eval_all_mutate(quo):
“NAs introduced by coercion”


age,sex,cp,trestbps,chol,fbs,restceg,max_heart_rate,exang,oldpeak,slope,ca,thal,heart_disease
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
63,1,1,145,233,1,2,150,0,2.3,3,0,6,False
67,1,4,160,286,0,2,108,1,1.5,2,3,3,True
67,1,4,120,229,0,2,129,1,2.6,2,2,7,True
37,1,3,130,250,0,0,187,0,3.5,3,0,3,False
41,0,2,130,204,0,2,172,0,1.4,1,0,3,False
56,1,2,120,236,0,0,178,0,0.8,1,0,3,False
62,0,4,140,268,0,2,160,0,3.6,3,2,3,True
57,0,4,120,354,0,0,163,1,0.6,1,0,3,False
63,1,4,130,254,0,2,147,0,1.4,2,1,7,True


In [3]:
options(repr.plot.width=22, repr.plot.height=6)


age_trestbps_plot <- heart_data |>
    ggplot(aes(x = age, y = trestbps)) +
        geom_point(aes(color = heart_disease), show.legend = FALSE) +
        labs (x = "Age (years)", y = "Resting Blood Pressure",  title = "Blood Pressure and Age Graph")+
        theme(text = element_text(size=17))

age_max_heart_rate_plot <- heart_data |>
    ggplot(aes(x = age, y = max_heart_rate)) +
        geom_point(aes(color = heart_disease), show.legend = FALSE) +
        labs (x = "Age (years)", y = "Maximum Heart Rate", title = "Maximum Heart Rate and Age Graph")+
        theme(text = element_text(size=17))

trestbps_max_heart_rate_plot <- heart_data |>
    ggplot(aes(x = trestbps, y = max_heart_rate)) +
        geom_point(aes(color = heart_disease)) +
        labs (x = "Resting Blood Pressure", y = "Maximum Heart Rate", color = "Has Heart Disease?", title = "Maximum Heart Rate and Blood Pressure")+
        theme(text = element_text(size=17))

trestbps_max_heart_rate_plot + age_max_heart_rate_plot + age_trestbps_plot + plot_layout(guides = 'collect')

ERROR: [1m[33mError[39m in [1m[1m`ggplot_add()`:[22m
[33m![39m Can't add `age_max_heart_rate_plot` to a ggplot object.


We already might be able to find some pattern between these variables but to better make predictions we will use a knn classification algorithm. In order to do that, the first step towards creating this algorithm is to split the data into training and testing data sets. 

In [4]:
set.seed(1)
heart_data <- heart_data |>
    select(age, trestbps, max_heart_rate, heart_disease)
heart_split <- initial_split(heart_data, prop = 0.75, strata = heart_disease)
heart_train <- training(heart_split)
heart_test <- testing(heart_split)

### Performing Classification with $k$-nearest algorithm, k=3

* First, we try with $k$=3 

In [5]:
set.seed(10)
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) |>
  set_engine("kknn") |>
  set_mode("classification")
heart_recipe <- recipe(heart_disease ~ age + trestbps + max_heart_rate, data= heart_train)|>
    step_scale(all_predictors())|>
    step_center(all_predictors())
heart_fit <- workflow()|>
    add_recipe(heart_recipe)|>
    add_model(knn_spec)|>
    fit(data = heart_train)
heart_fit


ERROR: [1m[33mError[39m in [1m[1m`check_installs()`:[22m
[33m![39m This engine requires some package installs: 'kknn'


#### Now, we perform 10-fold cross-validation to get the best $k$ value possible

In [ ]:
set.seed(100)
options(repr.plot.width=15, repr.plot.height=6)

heart_vfold <- vfold_cv(heart_train, v = 10, strata = heart_disease)
knn_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

heart_resamples_fit <- workflow()|>
    add_recipe(heart_recipe)|>
    add_model(knn_best_spec)|>
    tune_grid(resamples = heart_vfold, grid=20)|>
    collect_metrics()|>
    filter(.metric == "accuracy")

accuracy_v_k_plot <- heart_resamples_fit |>
    ggplot(aes(x= neighbors, y= mean))+
        geom_point()+
        geom_line()+
        labs(x= "K Value", y= "Accuracy Estimate")+
        theme(text = element_text(size=20))
accuracy_v_k_plot

* We observe from the graph above that the best $k$ values are $k=9$ or $k=10$. For this classification, we will chose $k=10$. Now, we perform $k$-nearest neighbors classification with $k=10$. 

In [ ]:
knn_spec_10 <- nearest_neighbor(weight_func = "rectangular", neighbors = 10) |>
  set_engine("kknn") |>
  set_mode("classification")

heart_best_fit <- workflow()|>  
    add_recipe(heart_recipe)|>
    add_model(knn_spec_10)|>
    fit(data = heart_train)

heart_best_fit

In [ ]:
heart_test_predections <- predict(heart_best_fit, heart_test)|>
    bind_cols(heart_test)
head(heart_test_predections, n=10)

In [ ]:
heart_test_accuracy <- heart_test_predections |>
    metrics(truth = heart_disease, estimate = .pred_class)
heart_test_accuracy

In [ ]:
heart_matrix <- heart_test_predections |>
    conf_mat(truth = heart_disease, estimate = .pred_class)
heart_matrix

We have $55$ correctly labelled values.

In [ ]:
wkflw_plot <-
  ggplot() +
  geom_point(data = heart_data, 
             mapping = aes(x = age, 
                           y = max_heart_rate, 
                           color = heart_disease), 
             alpha = 0.75) +
  geom_point(data = heart_test_predections, 
             mapping = aes(x = age, 
                           y = max_heart_rate, 
                           color = heart_disease), 
             alpha = 0.02, 
             size = 5) +
  labs(color = "Diagnosis", 
       x = "Area (standardized)", 
       y = "Smoothness (standardized)") +
  scale_color_manual(labels = c("Malignant", "Benign"), 
                     values = c("orange2", "steelblue2")) +
  theme(text = element_text(size = 12))



wkflw_plot

tried
